<div style="width: 100%; margin: 0 auto;">
    <a href="https://github.com/e10101/learning-artificial-intelligence">
        <img src="../assets/banner.svg" alt="Learning Operations Research" style="width: 100%; height: auto; display: block;">
    </a>
</div>

# Model Context Protocol (MCP) - Exchange Rate API Demo
---

[![Github](../assets/badges/github.svg)](https://github.com/e10101/learning-artificial-intelligence)


For this MCP demo, we will show to setup an Exchange Rate API MCP server and use it in a MCP client.

## Exchange Rate API

For simplicity, we will use existing Exchange Rate API service for example the [Exchange-Rate API](https://www.exchangerate-api.com/).
We choose this because it contains free budget (1,500 API calls) and easy to use.

You can [Sign Up](https://app.exchangerate-api.com/) here to get a API Key for making request to get exchange rates.

### Endpoints

For getting rates based on one currency, you can use following endpoint:

`https://v6.exchangerate-api.com/v6/<your_api_key>/latest/USD`

(Please replace the `<your_api_key>` with yours)

The example payload will be:

```json5
{
 "result":"success",
 "documentation":"https://www.exchangerate-api.com/docs",
 "terms_of_use":"https://www.exchangerate-api.com/terms",
 "time_last_update_unix":1745366402,
 "time_last_update_utc":"Wed, 23 Apr 2025 00:00:02 +0000",
 "time_next_update_unix":1745452802,
 "time_next_update_utc":"Thu, 24 Apr 2025 00:00:02 +0000",
 "base_code":"USD",
 "conversion_rates":{
  "USD":1,
  "AED":3.6725,
  "AFN":71.6759,
  "ALL":86.3697,
  "AMD":391.5106,
  "ANG":1.7900,
  ...
  "ZAR":18.6188,
  "ZMW":28.6439,
  "ZWL":26.8089
 }
}
```

Therefore, with above JSON payload, we can answer questions like:

- How much is 100 Euros (EUR) worth in Japanese Yen (JPY) right now?
- If I have 500 British Pounds (GBP), how many Canadian Dollars (CAD) can I get?
- ...

## Wrap the API with Python

### Install `uv`

For simplify our work, we will install `uv` first as the MCP official website suggests.
Please use following command:

```shell
curl -LsSf https://astral.sh/uv/install.sh | sh
```

### Create project

Then we can create a new project to work with for this demo:

```shell
# Create a new project with uv
uv init mcp-server
cd mcp-server

# Create a virtual environment add activate it
uv venv
source .venv/bin/activate

# Install pendencies
uv add "mcp[cli]" httpx
```

### Write codes

Open the `main.py` file in the `mcp-server` folder,
replace the content with following codes:

```python
# Import packages
import os
import httpx
from mcp.server.fastmcp import FastMCP

# Create variables about calling the 3rd party API.
BASE_URL = os.getenv("BASE_URL", "https://v6.exchangerate-api.com/v6")
API_KEY = os.getenv("API_KEY")

# Create a mcp server
mcp = FastMCP("Exchange Rate Server")

# Define a tool about getting latest rates of one currency.
@mcp.tool()
async def get_latest_rates(base_currency: str) -> dict:
    """
    Get the latest exchange rates for a given base currency.

    Args:
        base_currency (str): The base currency (e.g. USD, EUR, GBP, etc.) to get the latest rates for.

    Returns:
        dict: A dictionary containing the latest exchange rates.
    """

    url = f"{BASE_URL}/{API_KEY}/latest/{base_currency}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        response.raise_for_status()

        return response.json()

# Run codes
if __name__ == "__main__":
    mcp.run(transport="stdio")
```

## Testing

After we finishing the coding part, we can now to debug the tool:

```shell
# Run the command in the mcp-server folder
mcp dev main.py
```

The `modelcontextprotocol/inspector@0.10.2` will running at [http://127.0.0.1:6274](http://127.0.0.1:6274).

Put your `API_KEY` in the (Environment Variables) config part,
now you can (Connect) -> (Tools) -> (List Tools) -> (Input a `base_currency`, for example `CNY`) -> (Run Tool).

If all good, you will see the API response displayed in the output box.

## Install for Claude Desktop

If we can get successful response from the API server, then we can now try to install the created MCP server to a MCP client (for example Cluade Desktop App).

We can run command to install directly to Claude Desktop:

```shell
mcp install main.py
```

For MacOS users, please don't forget to add your `API_KEY` to the claude config JSON file
(`/Users/<user_name>/Library/Application Support/Claude/claude_desktop_config.json`).
The config should be something like:

```json
{
  "mcpServers": {
    "Exchange Rate Server": {
      "command": "/Users/<user_name>/.local/bin/uv",
      "args": [
        "run",
        "--with",
        "mcp[cli]",
        "mcp",
        "run",
        "/Users/<user_name>/<some_path>/mcp-server/main.py"
      ],
      "env": {
        "API_KEY": "<your_api_key>"
      }
    }
  }
}
```

Please don't forget to replace the variables to your real ones.

After restarting Claude Desktop, you will see there is a new Tools available:

![Claude Desktop MCP Tools](./images/claude-mcp-tools.png)

And in the settings panel, you can also see our installed tools:

![Claude Desktop MCP Tools Installed](./images/claude-mcp-installed.png)

## Using MCP

After installing successfully, we can ask questions like:

```
How much is 100 Euros worth in Japanese Yen right now?
```

And get answers with our created MCP server:

```
To answer your question about the current exchange rate between Euros and Japanese Yen, I'll need to look up the latest rates. Let me do that for you.

Based on the latest exchange rates, 100 Euros (€100) is currently worth approximately 16,169 Japanese Yen (¥16,169).

The exact exchange rate is €1 = ¥161.69, as of today, April 23, 2025.

Keep in mind that exchange rates fluctuate continuously, and there may be additional fees if you're planning to convert currency through a bank or exchange service.
```

Screenshot like:

![Claude Desktop with MCP result](./images/claude-mcp-qna.png)

## Conclusion

In this demo, we have shown how to create a simple MCP server with Python to get Exchange Rates information and use it in a MCP client (Claude Desktop).

The source code of this demo can be found at [e10101/learning-artificial-intelligence](https://github.com/e10101/learning-artificial-intelligence).

## References

- https://modelcontextprotocol.io/quickstart/server
- https://www.exchangerate-api.com/docs/standard-requests